# EJERCICIOS

In [1]:
import warnings
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier,StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

## EJERCICIO 1

Si has entrenado cinco modelos diferentes en el mismo conjunto de entrenamiento exacto y todos consiguen una precisión del 95%, ¿hay alguna posibilidad de que puedas combinar estos modelos para obtener mejores resultados? 

Si la respuesta es sí, ¿cómo? Si la respuesta es no, ¿por qué?

In [ ]:
# Si, en un conjunto de votación, aun que todos fallen en un 5% no quiere decir que todos fallen en el mismo 5%,
# donde cada uno falla los demas acertarán aumentando ese 95%. 

## EJERCICIO 2

Carga el conjunto de datos MNIST y divídelo en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba (por ejemplo, utiliza 50.000 instancias para entrenamiento, 10.000 para validación y 10.000 para pruebas). 

Después, entrena varios clasificadores diferentes (uno de ellos que sea un árbol de decisión). 

A continuación, intenta combinarlos en un ensamble que supere en rendimiento a cada clasificador individual del conjunto de validación, utilizando hard voting. 

Una vez que hayas encontrado uno, pruébalo en el conjunto de pruebas.

In [4]:
mnist = fetch_openml('mnist_784', as_frame=False, parser="auto" )
X_mnist, y_mnist = mnist.data, mnist.target

# Cogemos los primeros 50.000 para entrenar
X_train, y_train = X_mnist[:50_000], y_mnist[:50_000]

# Dividimos el resto
X_valid, y_valid = X_mnist[50_000:60_000], y_mnist[50_000:60_000]
X_test, y_test = X_mnist[60_000:], y_mnist[60_000:]

In [6]:
dec_tree = DecisionTreeClassifier(random_state=17)
knnc = KNeighborsClassifier()
svc = SVC(random_state=17)

classifiers = [dec_tree, knnc, svc]
for classifier in classifiers:
    print("Entrenando ", classifier.__class__.__name__)
    classifier.fit(X_train, y_train)
    print(classifier.score(X_valid, y_valid))

Entrenando  DecisionTreeClassifier
0.8797
Entrenando  KNeighborsClassifier
0.9718
Entrenando  SVC
0.9802


In [8]:
[classifier.score(X_valid, y_valid) for classifier in classifiers]

[0.8797, 0.9718, 0.9802]

In [9]:
named_classifiers = [
    ("dec_tree", dec_tree),
    ("knnc", knnc),
    ("svc", svc)
]

voting = VotingClassifier(named_classifiers)
error_train=voting.fit(X_train, y_train)
error_val=voting.score(X_valid, y_valid)
print("Precisión del conjunto de entrenamiento: ", error_train)
print("Precisión del conjunto de validación: ", error_val)

Precisión del conjunto de entrenamiento:  VotingClassifier(estimators=[('dec_tree',
                              DecisionTreeClassifier(random_state=17)),
                             ('knnc', KNeighborsClassifier()),
                             ('svc', SVC(random_state=17))])
Precisión del conjunto de validación:  0.9755


In [ ]:
voting_score = voting.score(X_test, y_test)
print("Precisión del ensamble:",voting_score)

Precisión del ensamble:0.9721


## EJERCICIO 3

Ejecuta los clasificadores individuales del ejercicio anterior para hacer predicciones en el conjunto de entrenamiento y crea un nuevo conjunto de entrenamiento con las predicciones resultantes: cada instancia de entrenamiento es un vector que contiene el conjunto de predicciones de todos tus clasificadores para una imagen y el objetivo es la clase de la imagen. Entrena un clasificador (RandomForestClassifier) en este nuevo conjunto de entrenamiento. 

Acabas de entrenar un blender y, junto a los clasificadores, forma un ensamble de stacking.

Ahora, evalúa el ensamble en el conjunto de prueba. 

¿Cómo es en comparación con el clasificador de votación que has entrenado antes?

Haz lo mismo usando StackingClassifier

In [11]:
# Generar vectores vacíos
X_train_pred = np.empty((len(X_train), len(classifiers)), dtype=object)

for index, classifier in enumerate(classifiers):
    X_train_pred[:, index] = classifier.predict(X_train)

# Creamos el blender y lo entrenamos
rnd_forest_blender = RandomForestClassifier(n_estimators=100, random_state=17)
rnd_forest_blender.fit(X_train_pred, y_train)

RandomForestClassifier(random_state=17)

In [12]:
# Generar vectores vacíos
X_test_pred = np.empty((len(X_test), len(classifiers)), dtype=object)

for index, classifier in enumerate(classifiers):
    X_test_pred[:, index] = classifier.predict(X_test)

y_pred = rnd_forest_blender.predict(X_test_pred)

# Lo probamos en el conjunto de pruebas
acc_score = accuracy_score(y_test, y_pred)
print("Precisión del conjunto de prueba:", acc_score)

Precisión del conjunto de prueba: 0.8742


In [13]:
# Creamos el blender y lo entrenamos
stack_clf = StackingClassifier(named_classifiers,final_estimator=rnd_forest_blender)
stack_clf.fit(X_train, y_train)

# Lo probamos en el conjunto de pruebas
stack_clf.score(X_test, y_test)

0.9801